In [10]:
!pip install numpy
!pip install pygame


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import numpy as np

# 8x8 spēles laukums kā divdimensiju masīvs, kur "e" ir tukša rūtiņa un "w", "b" ir baltās vai melnās figūras.

EMPTY = "e"
WHITE = "w"
BLACK = "b"

class Board:
    def __init__(self):
        self.board=np.full(shape=(8,8), fill_value=EMPTY)
        self.board[4][4] = WHITE
        self.board[4][3] = BLACK
        self.board[3][3] = WHITE
        self.board[3][4] = BLACK

GAME_STATE = True

# Sataksts ar visiem iespējamiem gājiena virzieniem
DIRECTIONS = []
for i in range(-1, 2):
    for j in range(-1, 2):
        DIRECTIONS.append([i,j])
DIRECTIONS.remove([0,0])

# Funkcija, kas atgriež pretējo krāsu iedotai
def opposite_color(color):
    if color == BLACK:
        return WHITE
    else:
        return BLACK

# Funkcija, kas pārbauda vai gājiens ir iespējams.
def is_valid_move(board, player, row, col):
    if board[row][col] != 'e':
        return False

    for direction in DIRECTIONS:
        i, j = direction[0], direction[1]
        r, c = row + i, col + j
        if r < 0 or r >= 8 or c < 0 or c >= 8:
            continue
        if board[r][c] == opposite_color(player):
            while True:
                r, c = r + i, c + j
                if r < 0 or r >= 8 or c < 0 or c >= 8:
                    break
                if board[r][c] == player:
                    return True
                if board[r][c] == 'e':
                    break

    return False

# Funkcija, kas atgriež iespējamos gājienus visā spēles laukā.
def get_valid_moves(board, player):
    valid_moves = []
    for row in range(8):
        for col in range(8):
            if is_valid_move(board, player, row, col):
                valid_moves.append((row, col))
    return valid_moves


# Funkcija kas pārbauda visos virzienos kuras figūras ir jāmaina.
# Ja satiek uz kādu virzienu savas krāsas figūru tad visas figūras starp kas 
# uz laiku ir saglabātas "pieces_to_flip" tiek nomainītas uz savējiem.
def flip_pieces(board, player, row, col):
    for direction in DIRECTIONS:
        r, c = row + direction[0], col + direction[1]
        if r < 0 or r >= 8 or c < 0 or c >= 8:
            continue
        if board[r][c] == player or board[r][c] == EMPTY:
            continue
        pieces_to_flip = []
        while board[r][c] != player:
            pieces_to_flip.append((r, c))
            r += direction[0]
            c += direction[1]
            if r < 0 or r >= 8 or c < 0 or c >= 8 or board[r][c] == EMPTY:
                break
            else:
                for piece in pieces_to_flip:
                    board[piece[0]][piece[1]] = player
    board[row][col] = player

# Funkcija uzvarētāja noreikšanai
def get_winner(board):
    white_count = np.count_nonzero(board == WHITE)
    black_count = np.count_nonzero(board == BLACK)
    if white_count > black_count:
        return WHITE
    elif black_count > white_count:
        return BLACK
    else:
        return EMPTY

# Funkcija kas izpilda soli pārbaudot vai tas ir spēles laukuma robežās (grafiska interfeisa kļūda)
# un pārbauda vai tas ir iespējams solis. Tad to īsteno.
def make_move(board, player, row, col):
    if row < 0 or row >= len(board):
        return False
    if col < 0 or col >= len(board[0]):
        return False
    if not is_valid_move(board, player, row, col):
        return False
    flip_pieces(board, player, row, col)
    return True

# Funkcija kas saskaita baltas un melnas figūras uz lauka atspoguļojuma
# divdimensiju masīvā.
def get_score(board):
    black_count = 0
    white_count = 0
    for row in range(8):
        for col in range(8):
            if board[row][col] == 'b':
                black_count += 1
            elif board[row][col] == 'w':
                white_count += 1
    
    return (black_count, white_count)

import pygame
import sys

WINDOW_WIDTH = 450
WINDOW_WIDTH_CENTER = 450 // 2
WINDOW_HEIGHT = 500
WINDOW_SIZE = [WINDOW_WIDTH, WINDOW_HEIGHT]
BLACK_COLOR = (0, 0, 0)
WHITE_COLOR = (255, 255, 255)
GREEN_COLOR = (0, 255, 0)
RED_COLOR = (255, 0, 0)

SQUARE_SIZE = 56.25
HALF_SQUARE_SIZE = 56.25 // 2

WIDTH = 50
HEIGHT = 50
MARGIN = 5

current_player = BLACK
clock = pygame.time.Clock()
pygame.init()
screen = pygame.display.set_mode(WINDOW_SIZE)
font = pygame.font.SysFont(None, 24)

pygame.display.set_caption("SalaspilsVersi Game")

done = False

# Spēles laukuma sadalīšana rūtiņās
def draw_lines():
    for i in range(8):
        start_pos = (0, i * SQUARE_SIZE + SQUARE_SIZE )
        end_pos = (450, i * SQUARE_SIZE + SQUARE_SIZE)
        pygame.draw.line(screen, BLACK_COLOR, start_pos, end_pos)

    for i in range(8):
        start_pos = (i * SQUARE_SIZE + SQUARE_SIZE, 0)
        end_pos = (i * SQUARE_SIZE + SQUARE_SIZE, 450)
        pygame.draw.line(screen, BLACK_COLOR, start_pos, end_pos)

# Funkcija kas pārbauda vai spēle ir beigusies.
def is_game_over(board, current_player):
    black_moves = get_valid_moves(board, BLACK)
    white_moves = get_valid_moves(board, WHITE)
    if current_player == "b" and len(black_moves) == 0:
        return True
    elif current_player == "w" and len(white_moves) == 0:
        return True
    else:
        return False

# Funkcija kas parāda uzvarētāju.
def show_winner(screen, winner):
    font = pygame.font.Font(None, 50)
    text = font.render(f"Winner: {winner}", True, RED_COLOR)
    text_rect = text.get_rect(center=(WINDOW_WIDTH_CENTER, 50))
    screen.blit(text, text_rect)

# Funkcija kas izrēķina spēlētāja stāvokļa heiristisko vērtējumu.
def get_score_minmax(board, player):
    weights = np.array([
        [100, -3, 1, 1, 1, 1, -3, 100],
        [-3, -5, -1, -1, -1, -1, -5, -3],
        [10, -1, 1, 1, 1, 1, -1, 10],
        [8, -1, 1, 1, 1, 1, 5, 8],
        [8, -1, 1, 1, 1, 1, -1, 8],
        [10, -1, 1, 1, 1, 1, -1, 10],
        [-3, -5, -1, -1, -1, -1, -5, -3],
        [100, -3, 1, 1, 1, 1, -3, 100]
    ])

    player_score = np.sum(weights*(board == player))
    opponent_score = np.sum(weights*(board == opposite_color(player)))
    return player_score - opponent_score
 
#Zīmē galveno izvēles ekrānu
def show_menu(screen):
    screen.fill(BLACK_COLOR)
    font = pygame.font.Font(None, 36)
    title_text = font.render("Menu", True, WHITE_COLOR)
    new_game_text = font.render("Against player", True, WHITE_COLOR)
    ai1_game_text = font.render("Against AI start 1", True, WHITE_COLOR)
    ai2_game_text = font.render("Against AI start 2", True, WHITE_COLOR)
    quit_text = font.render("Quit", True, WHITE_COLOR)

    title_rect = title_text.get_rect(center=(WINDOW_WIDTH_CENTER, 100))
    new_game_rect = new_game_text.get_rect(center=(WINDOW_WIDTH_CENTER, 200))

    ai1_game_rect = ai1_game_text.get_rect(center=(WINDOW_WIDTH_CENTER, 250))
    ai2_game_rect = ai2_game_text.get_rect(center=(WINDOW_WIDTH_CENTER, 300))

    quit_rect = quit_text.get_rect(center=(WINDOW_WIDTH_CENTER, 350))

    screen.blit(title_text, title_rect)
    screen.blit(new_game_text, new_game_rect)
    screen.blit(quit_text, quit_rect)

    screen.blit(ai1_game_text, ai1_game_rect)
    screen.blit(ai2_game_text, ai2_game_rect)
    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                if new_game_rect.collidepoint(mouse_pos):
                    return '1'
                elif quit_rect.collidepoint(mouse_pos):
                    return '2'
                elif ai1_game_rect.collidepoint(mouse_pos):
                    return '3'
                elif ai2_game_rect.collidepoint(mouse_pos):
                    return '4'

# Vizualizē spēlētāju figūru daudzumu zem spēles laukuma.    
def render_score(surface, font, score):
    if current_player == "b":
        player = "Black"
    else:
        player = "White"
    black_score, white_score = score
    text = f"Black score: {black_score}   White score: {white_score} Now: {player}"
    text_surface = font.render(text, True, (0, 0, 0))
    surface.blit(text_surface, (10, 450))

# Vizualuzē figūru novietojumu uz spēļu laukuma.
def draw_board(screen, board):
    for row in range(8):
        for col in range(8):
            x = col * SQUARE_SIZE
            y = row * SQUARE_SIZE
            if board[row][col] == BLACK:
                pygame.draw.circle(screen, BLACK_COLOR, (x+HALF_SQUARE_SIZE, y+HALF_SQUARE_SIZE), HALF_SQUARE_SIZE - 5)
            elif board[row][col] == WHITE:
                pygame.draw.circle(screen, WHITE_COLOR, (x+HALF_SQUARE_SIZE, y+HALF_SQUARE_SIZE), HALF_SQUARE_SIZE - 5)

# Minimaksa algoritms apskata sevī visus turpmākos iespējamos gājienus un gājienus aiz tiem n
# dziļumā. Salīdzina tos ar heiristiskās novērtējuma funkcijas palīdzību un atgriež labāko gājienu.

def minimax(board, depth, player, max_player, min_player):
    if depth == 0:
        return None, get_score_minmax(board, player)
    
    if player == max_player:
        best_score = float("-inf")
        best_move = None
        for move in get_valid_moves(board, player):
            new_board = np.copy(board)
            make_move(new_board, player, move[0], move[1])
            score = minimax(new_board, depth-1, opposite_color(player), max_player, min_player)[1]
            if score > best_score:
                best_score = score
                best_move = move
        return best_move, best_score
    
    else:
        best_score = float("inf")
        best_move = None
        for move in get_valid_moves(board, player):
            new_board = np.copy(board)
            make_move(new_board, player, move[0], move[1])
            score = minimax(new_board, depth-1, opposite_color(player), max_player, min_player)[1]
            if score < best_score:
                best_score = score
                best_move = move
        return best_move, best_score

# Galvenais spēles kods
while True:
    # Zīmē galveno izvēles ekranu
    option = show_menu(screen)
    # Spēles režīms spēlētājs pret spēlētāju
    if option == '1':
        board = Board()
        current_player = 'b'
        winner = None
        done=False

        while not done:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    if make_move(board.board, current_player, row, column):
                        current_player = opposite_color(current_player)

            if is_game_over(board.board, current_player):
                black_score, white_score = get_score(board.board)
                if black_score > white_score:
                    winner = "Black"
                elif white_score > black_score:
                    winner = "White"
                else:
                    winner = "Tie"

            screen.fill(GREEN_COLOR)
            font = pygame.font.Font(None, 24)
            button_text = font.render("Return to Menu", True, BLACK_COLOR)
            button_rect = button_text.get_rect(center=(WINDOW_WIDTH_CENTER, 475))
            screen.blit(button_text, button_rect)
            mouse_pos = pygame.mouse.get_pos()
            for event in pygame.event.get():
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if button_rect.collidepoint(mouse_pos):
                        done = True

            draw_lines()

            draw_board(screen, board.board)

            render_score(screen, font, get_score(board.board))

            if winner:
                show_winner(screen, winner)

            pygame.display.flip()

            clock.tick(60)

    # Quit izvēle
    elif option == '2':
        pygame.quit()
    
    # Spēle pret AI, AI pirmais
    elif option == '3':
        board = Board()
        current_player = 'b'
        winner = None
        done=False

        while not done:


            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                elif current_player == BLACK:
                    move, score = minimax(board.board, 4, current_player, BLACK, WHITE)

                    if move == None:
                        move = get_valid_moves(board.board, BLACK)[0]
                        
                    make_move(board.board, current_player, move[0], move[1])
                    current_player = opposite_color(current_player)
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    if current_player==WHITE and make_move(board.board, current_player, row, column):
                        current_player = opposite_color(current_player)

            if is_game_over(board.board, current_player):
                black_score, white_score = get_score(board.board)

                if black_score > white_score:
                    winner = "Black"
                elif white_score > black_score:
                    winner = "White"
                else:
                    winner = "Tie"

            screen.fill(GREEN_COLOR)
            font = pygame.font.Font(None, 24)
            button_text = font.render("Return to Menu", True, BLACK_COLOR)
            button_rect = button_text.get_rect(center=(WINDOW_WIDTH_CENTER, 475))
            screen.blit(button_text, button_rect)
            mouse_pos = pygame.mouse.get_pos()
            for event in pygame.event.get():
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if button_rect.collidepoint(mouse_pos):
                        done = True

            draw_lines()

            draw_board(screen, board.board)


            render_score(screen, font, get_score(board.board))

            if winner:
                show_winner(screen, winner)

            pygame.display.flip()

            clock.tick(60)

     # Spēle pret AI, AI otrais
    elif option == '4':
        board = Board()
        current_player = 'b'
        winner = None
        done = False

        while not done:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                elif current_player == WHITE:
                    move, score = minimax(board.board, 4, current_player, WHITE, BLACK)
                    print(move, score)
                    make_move(board.board, current_player, move[0], move[1])
                    current_player = opposite_color(current_player)
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    if current_player==BLACK and make_move(board.board, current_player, row, column):
                        current_player = opposite_color(current_player)

            if is_game_over(board.board, current_player):
                black_score, white_score = get_score(board.board)

                if black_score > white_score:
                    winner = "Black"
                elif white_score > black_score:
                    winner = "White"
                else:
                    winner = "Tie"

            screen.fill(GREEN_COLOR)
            font = pygame.font.Font(None, 24)
            button_text = font.render("Return to Menu", True, BLACK_COLOR)
            button_rect = button_text.get_rect(center=(WINDOW_WIDTH_CENTER, 475))
            screen.blit(button_text, button_rect)
            mouse_pos = pygame.mouse.get_pos()
            for event in pygame.event.get():
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if button_rect.collidepoint(mouse_pos):
                        done = True
            draw_lines()

            draw_board(screen, board.board)

            render_score(screen, font, get_score(board.board))

            if winner:
                show_winner(screen, winner)

            pygame.display.flip()

            clock.tick(60)

    else:
        pygame.quit()
pygame.quit()


error: display Surface quit

In [16]:
move_dir = []
for i in range(-1, 2):
    for j in range(-1, 2):
        move_dir.append([i,j])
move_dir.remove([0,0])

print(move_dir[1][1])

0
